In [3]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(parallel)
library(jsonlite)



In [4]:
meta_data=read.csv("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv")
meta_data_subset<-meta_data %>% filter(Covid_exclusion=='no',
                                       sample.visitName%in% c('Flu Year 1 Day 0'))

In [5]:
aggregated_count_file_list<-paste0("/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA/BRI/Average_LogNormalized_Expression/Average_LogNormalized_Expression_of_Celltypes_by_Sample_AIFI_L3/",meta_data_subset$pbmc_sample_id,".csv")
df_list<-mclapply(aggregated_count_file_list,function(x){
df<-read.csv(x,check.names = FALSE, row.names = 1)
colnames(df)<-paste0(gsub("^.*/(.*)\\.csv$", "\\1", x),":",colnames(df))
return (df)
},mc.cores=60)

# Read Top Genes in NMF

In [6]:
NMF_dict <- fromJSON('NMF_dict_Top20.json')

In [7]:
celltype_list <- lapply(df_list, function(df) { df <- df[NMF_dict[['NMF6_Tfh']][1:20], grep("CM CD4 T cell", names(df), fixed = TRUE),drop=FALSE] })
exp_matrix<-do.call(cbind,celltype_list)

In [8]:
colnames(exp_matrix)<-sub(":.*", "", colnames(exp_matrix))
rownames(meta_data_subset)<-meta_data_subset$pbmc_sample_id
write.csv(exp_matrix,'Top20_Gex_Tfh_NMF.csv')